In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  #

W1007 17:19:10.104164 4628436416 __init__.py:690] 

  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  
W1007 17:19:11.436979 4628436416 deprecation.py:323] From <ipython-input-1-339a9f4bb712>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1007 17:19:11.437775 4628436416 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a futur

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W1007 17:19:11.662976 4628436416 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [18]:
class Config:
    def __init__(self):
        self.n_input = 784  # input layer (28x28 pixels)
        self.n_hidden1 = 512  # 1st hidden layer
        self.n_hidden2 = 256  # 2nd hidden layer
        self.n_hidden3 = 128  # 3rd hidden layer
        self.n_output = 10  # output layer (0-9 digits)
        self.n_model_inputs = self.n_input + self.n_hidden3  # input layer (28x28 pixels)

        self.learning_rate = 1e-4
        self.learn_iterations = 1000
        self.batch_size = 200
        self.dropout = 0.5

class Mach:

    def __init__(self, config, mnist):
        self.c = config
        self.mnist = mnist
        self.n_train = self.mnist.train.num_examples  # 55,000
        self.n_validation = self.mnist.validation.num_examples  # 5000
        self.n_test = self.mnist.test.num_examples  # 10,000

        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)
        with self.graph.as_default():
            self.input_fn()
            self.model_fn()
            self.loss_fn()
            init = tf.global_variables_initializer()
        self.session.run(init)
        
        self.child = None
        
    def input_fn(self):
        # Placeholders.
        self.X = tf.placeholder(tf.float32, [None, self.c.n_input])
        self.Y = tf.placeholder(tf.float32, [None, self.c.n_output])
        self.C = tf.placeholder(tf.float32, [None, self.c.n_hidden3])
        self.keep_prob = tf.placeholder(tf.float32)
            
    def loss_fn(self):
        self.cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.Y, logits=self.logits))
        self.train_step = tf.train.AdamOptimizer(self.c.learning_rate).minimize(self.cross_entropy)
        self.correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))
    
    def model_fn(self):
        self.model_inputs = tf.concat([self.X, self.C], axis=1)
        
        weights = {
            'w1': tf.Variable(tf.truncated_normal([self.c.n_model_inputs, self.c.n_hidden1], stddev=0.1)),
            'w2': tf.Variable(tf.truncated_normal([self.c.n_hidden1, self.c.n_hidden2], stddev=0.1)),
            'w3': tf.Variable(tf.truncated_normal([self.c.n_hidden2, self.c.n_hidden3], stddev=0.1)),
            'out': tf.Variable(tf.truncated_normal([self.c.n_hidden3, self.c.n_output], stddev=0.1)),
        }

        biases = {
            'b1': tf.Variable(tf.constant(0.1, shape=[self.c.n_hidden1])),
            'b2': tf.Variable(tf.constant(0.1, shape=[self.c.n_hidden2])),
            'b3': tf.Variable(tf.constant(0.1, shape=[self.c.n_hidden3])),
            'out': tf.Variable(tf.constant(0.1, shape=[self.c.n_output]))
        }

        self.layer_1 = tf.add(tf.matmul(self.model_inputs, weights['w1']), biases['b1'])
        self.layer_2 = tf.add(tf.matmul(self.layer_1, weights['w2']), biases['b2'])
        self.layer_3 = tf.add(tf.matmul(self.layer_2, weights['w3']), biases['b3'])
        layer_drop = tf.nn.dropout(self.layer_3, rate = (1 - self.keep_prob))
        self.logits = tf.matmul(self.layer_3, weights['out']) + biases['out']
        
    def Child(self, batch):
        if self.child == None:
            return numpy.zeros((numpy.shape(batch)[0], self.c.n_hidden3))
        else:
            return self.child.Spike(batch)
        
    def Spike(self, batch):
        feeds={self.X: batch, self.C: self.Child(batch), self.keep_prob: 1.0}
        return self.session.run([self.layer_3], feed_dict=feeds)[0]
        
    def Train(self):
        # train on mini batches
        for i in range(self.c.learn_iterations):    
            batch_x, batch_y = self.mnist.train.next_batch(self.c.batch_size)
            feeds={self.X: batch_x, self.Y: batch_y, self.C: self.Child(batch_x), self.keep_prob: self.c.dropout}
            self.session.run(self.train_step, feed_dict=feeds)

#             # print loss and accuracy (per minibatch)
#             if i % 100 == 0:
#                 fetches = [self.cross_entropy, self.accuracy]
#                 feeds = {self.X: batch_x, self.Y: batch_y, self.C: self.Child(batch_x), self.keep_prob: 1.0}
#                 minibatch_loss, minibatch_accuracy = self.session.run(fetches,feeds)    
#                 print(
#                     "Iteration",
#                     str(i),
#                     "\t| Loss =",
#                     str(minibatch_loss),
#                     "\t| Accuracy =",
#                     str(minibatch_accuracy)
#                     )
                
    def Test(self):
        feed_dict = {
                self.X: self.mnist.test.images, 
                self.Y: self.mnist.test.labels, 
                self.C: self.Child(self.mnist.test.images), 
                self.keep_prob: 1.0
        }
        test_accuracy = self.session.run(self.accuracy, feed_dict=feed_dict)
        print("\nAccuracy on test set:", test_accuracy)



In [19]:
c = Config()
m = Mach(c, mnist)

c2 = Config()
m2 = Mach(c2, mnist)

m.child = m2

In [21]:
for _ in range(1000):
    m2.Train()
    m.Train()
    m.Test()


Accuracy on test set: 0.9194

Accuracy on test set: 0.9234

Accuracy on test set: 0.9229

Accuracy on test set: 0.9215

Accuracy on test set: 0.9231

Accuracy on test set: 0.9204

Accuracy on test set: 0.9208

Accuracy on test set: 0.9215

Accuracy on test set: 0.9205

Accuracy on test set: 0.9217

Accuracy on test set: 0.9207

Accuracy on test set: 0.922

Accuracy on test set: 0.9194

Accuracy on test set: 0.9209

Accuracy on test set: 0.9213

Accuracy on test set: 0.9189

Accuracy on test set: 0.9192

Accuracy on test set: 0.9217

Accuracy on test set: 0.9209

Accuracy on test set: 0.9211

Accuracy on test set: 0.9211

Accuracy on test set: 0.9217

Accuracy on test set: 0.9205

Accuracy on test set: 0.9196

Accuracy on test set: 0.9214

Accuracy on test set: 0.9197

Accuracy on test set: 0.9224

Accuracy on test set: 0.9207

Accuracy on test set: 0.9202

Accuracy on test set: 0.9209

Accuracy on test set: 0.9212

Accuracy on test set: 0.9206

Accuracy on test set: 0.9217

Accuracy o

KeyboardInterrupt: 